In [ ]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import anthropic

import gradio as gr

In [ ]:
# Set Up Environment
load_dotenv(override=True)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
# Connect to Clients

openai = OpenAI()
claude = anthropic.Anthropic()

In [ ]:
# System Message

system_message = "You are a helpful assistant"

In [ ]:
# Call to GPT

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    completion = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
    )
    return completion.choices[0].message.content

In [ ]:
message_gpt("What is today's date?")

# Creating User Interface Using Gradio

In [ ]:
def shout(text):
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

In [ ]:
view = gr.Interface(
    fn = shout,
    inputs = [gr.Textbox(label="Your message:", lines=6)],
    outputs = [gr.Textbox(label="Response:", lines=8)],
    flagging_mode = "never"
)
view.launch()

In [ ]:
view = gr.Interface(
    fn = message_gpt,
    inputs = [gr.Textbox(label="Your message:", lines=6)],
    outputs = [gr.Textbox(label="Response:", lines=8)],
    flagging_mode = "never"
)
view.launch()

## Improving Gradio UI

In [ ]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn = message_gpt,
    inputs = [gr.Textbox(label="Your message:", lines=6)],
    outputs = [gr.Markdown(label="Response:")],
    flagging_mode = "never"
)
view.launch()

## Integrating Stream

In [ ]:
# Call to GPT using stream

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
system_message = "You are a helpful assistant that responds in markdown"

In [ ]:
view = gr.Interface(
    fn = stream_gpt,
    inputs = [gr.Textbox(label="Your message:")],
    outputs = [gr.Markdown(label="Response:")],
    flagging_mode = "never"
)
view.launch()

In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown Model")
    for chunk in result:
        yield chunk

In [ ]:
view = gr.Interface(
    fn = stream_model,
    inputs = [gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs = [gr.Markdown(label="Response:")],
    flagging_mode = "never"
)
view.launch()